In [1]:
from drn_interactions.io import load_recordings, load_neurons, load_derived_generic
from drn_interactions.stats import se_mean, se_prop, p_adjust, prop_of_total
from drn_interactions.spiketrains.neurontype_props import AnovaPostHoc, ChiSquarePostHoc
from IPython.display import display
import numpy as np
from scipy.stats import chi2_contingency
import pandas as pd
import pingouin as pg
from itertools import combinations

%load_ext autoreload
%autoreload 2

c:\Users\roryl\.conda\envs\drn-interactions\lib\site-packages\outdated\utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [2]:
def mean_se(x, round=1):
    m = np.mean(x)
    se = se_mean(x)
    return f"{m:2.{round}f} ± {se:2.{round}f}"

def mean_se_prop(x, round=2):
    p = np.mean(x) * 100
    se = se_prop(x) * 100
    return f"{p:2.{round}f} ± {se:2.{round}f}"


def compare_across_neuron_types(df, col, round=2):
    """
    Compare the mean and SE of a column across clusters.
    """
    df = df.copy()
    df["cluster"] = df["cluster"].astype(str)
    df = df.groupby(["cluster", col]).mean().reset_index()
    df = df.pivot(index="cluster", columns=col, values="mean")
    df.columns = [f"{col} {c}" for c in df.columns]
    df["se"] = df.apply(lambda x: se_mean(x), axis=1)
    df["prop"] = df.apply(lambda x: np.mean(x) * 100, axis=1)
    df["se_prop"] = df.apply(lambda x: se_prop(x) * 100, axis=1)
    df = df.round(round)
    return df


In [3]:
# number of recordings

df_recordings = load_recordings()

print(f"Total Recprdings: {len(df_recordings)}")


num_by_proto = df_recordings.groupby("experiment_name").apply(len).to_frame("Number of Recordings")
num_by_proto.index.name = "Recording Protocol"
display(num_by_proto)

Total Recprdings: 24


,Number of Recordings
Recording Protocol,
CITWAY,6
ESHOCK,8
HAMILTON,10


In [4]:
# number of neurons per recordings and overall

neurons = load_neurons()

total_neurons = len(neurons)
print(f"Total Neurons: {total_neurons}")

num_by_proto = neurons.groupby("experiment_name").apply(len).to_frame("Number of Neurons")
num_by_proto.index.name = "Recording Protocol"
display(num_by_proto)

num_by_session = neurons.groupby("session_name").apply(len)
print(f"Number of neurons per session: {mean_se(num_by_session)}")

Total Neurons: 566


,Number of Neurons
Recording Protocol,
CITWAY,158
ESHOCK,132
HAMILTON,276


Number of neurons per session: 25.7 ± 2.7


In [5]:
# proportion of units of each type

clusters = load_derived_generic("neuron_types.csv")
clusters = clusters.merge(neurons[["neuron_id", "session_name"]])

mean_yeild_per_session = (
    clusters.groupby(["session_name", "neuron_type"]).apply(len)
    .to_frame("N").reset_index()
    .groupby("neuron_type")["N"].apply(mean_se)
    .to_frame("Mean Yeild")
)
mean_yeild_per_session.index.name = "Neuron Type"
display(mean_yeild_per_session)


prop_total_by_session = (
    clusters
    .pivot_table(index="session_name", columns="neuron_type", values="neuron_id", aggfunc=len)
    .fillna(0)
    .transform(prop_of_total, axis=1)
    .reset_index()
    .melt(id_vars="session_name", var_name="neuron_type", value_name="Proportion of Total")
)
prop_total_by_session.groupby("neuron_type", as_index=False)["Proportion of Total"].apply(mean_se_prop)





,Mean Yeild
Neuron Type,
ff,4.7 ± 0.7
sir,10.4 ± 1.8
sr,11.0 ± 1.5


,neuron_type,Proportion of Total
0,ff,18.07 ± 8.20
1,sir,38.15 ± 10.36
2,sr,43.78 ± 10.58


In [6]:
df_single_unit = load_derived_generic("neuron_types_single_unit.csv").assign(**{"Electrode Type": "Single Unit"})
df_probe = load_derived_generic("neuron_types.csv").assign(**{"Electrode Type": "Silicon Probe"})

df = (
    pd.concat([df_single_unit[["Electrode Type", "neuron_type"]], df_probe[["Electrode Type", "neuron_type"]]])
    .pipe(lambda x: pd.crosstab(x["neuron_type"], x["Electrode Type"]))
)
stat, p, dof, expected = chi2_contingency(df)

props = df.apply(prop_of_total).round(2)
display(props)
star = "*" if p < 0.05 else ""
print(f"Single Unit v Probe Neuron Type, Chi Square Test: \n\tChi-Square({dof}) = {stat:.1f}\n\tp = {p:.2f}{star}")



Electrode Type,Silicon Probe,Single Unit
neuron_type,,
ff,0.17,0.13
sir,0.40,0.34
sr,0.43,0.53


Single Unit v Probe Neuron Type, Chi Square Test: 
	Chi-Square(2) = 4.6
	p = 0.10


In [12]:
clusters = load_derived_generic("neuron_types.csv").merge(load_neurons()[["neuron_id", "session_name"]])
spikes = load_derived_generic("spiketrain_stats_segments.csv")
waveforms = load_derived_generic("waveform_summary.csv")

meta_cols = ["neuron_id", "session_name", "neuron_type"]
spiketrain_stats = (
    spikes
    .merge(clusters)
    .merge(waveforms, how="outer")
    .melt(id_vars=meta_cols, var_name="metric", value_name="value")
    .dropna()
    .assign(value=lambda x: x.value.astype(float))
)

# numeric metrics
anova_calculator = AnovaPostHoc(neuron_type_col="neuron_type", value_col="value", round=1)
numeric_metrics = ["cv_isi", "cv_isi_burst", "mean_firing_rate", "median_burst_interval", "width_minpost",]
df_res_numeric = (
    spiketrain_stats
    .query(f"metric in @numeric_metrics")
    .groupby("metric")
    .apply(anova_calculator)
)

# categorical metrics
prop_calculator = ChiSquarePostHoc(neuron_type_col="neuron_type", value_col="value", round=1)
categorical_metrics = ["is_burst", "has_basepre", "has_basepost"]
df_res_categorical = (
    spiketrain_stats
    .query(f"metric in @categorical_metrics and value in (0, 1)")
    .groupby("metric")
    .apply(prop_calculator)
)

# display results
display(df_res_numeric)
display(df_res_categorical)

,F,ff - sir,ff - sr,sir - sr
metric,,,,
cv_isi,F(2)=137.8 (p=0.0*),0.6; 0.9 | T=-9.6 (p=0.0*),0.6; 0.6 | T=2.7 (p=0.0*),0.9; 0.6 | T=16.2 (p=0.0*)
cv_isi_burst,F(2)=281.9 (p=0.0*),0.5; 0.9 | T=-16.4 (p=0.0*),0.5; 0.5 | T=0.5 (p=0.9),0.9; 0.5 | T=22.2 (p=0.0*)
mean_firing_rate,F(2)=325.6 (p=0.0*),12.2; 2.1 | T=25.3 (p=0.0*),12.2; 4.2 | T=20.2 (p=0.0*),2.1; 4.2 | T=-6.8 (p=0.0*)
median_burst_interval,F(2)=6.9 (p=0.0*),0.0; 0.0 | T=2.1 (p=0.1),0.0; 0.0 | T=3.7 (p=0.0*),0.0; 0.0 | T=2.0 (p=0.1)
width_minpost,F(2)=115.7 (p=0.0*),50.3; 43.9 | T=3.1 (p=0.0*),50.3; 67.6 | T=-8.4 (p=0.0*),43.9; 67.6 | T=-14.9 (p=0.0*)


,anova,sir - ff,sir - sr,ff - sr
metric,,,,
has_basepre,Chi2(2)=2.4 (p=0.30),39.5%; 35.4% | Chi(1.0)=0.3 (p=0.7),39.5%; 32.0% | Chi(1.0)=2.1 (p=0.5),35.4%; 32.0% | Chi(1.0)=0.2 (p=0.7)
is_burst,Chi2(2)=88.8 (p=0.00*),6.7%; 35.9% | Chi(1.0)=41.1 (p=0.0*),6.7%; 2.5% | Chi(1.0)=3.8 (p=0.1),35.9%; 2.5% | Chi(1.0)=68.6 (p=0.0*)


In [7]:
# volitility of spike train properties by neuron type

spikes_vol = load_derived_generic("spiketrain_stats_volitility.csv")

meta_cols = ["neuron_id", "session_name", "neuron_type"]
spiketrain_vol = (
    spikes_vol
    .merge(clusters)
    .melt(id_vars=meta_cols, var_name="metric", value_name="value")
    .assign(value=lambda x: x.value.astype(float))
)

# numeric metrics
anova_calculator = AnovaPostHoc(neuron_type_col="neuron_type", value_col="value", round=1)
numeric_metrics = ["cv_isi", "cv_isi_burst", "mean_firing_rate", ]
df_res_numeric = (
    spiketrain_vol
    .query(f"metric in @numeric_metrics")
    .groupby("metric")
    .apply(anova_calculator)
)

# categorical metrics
prop_calculator = ChiSquarePostHoc(neuron_type_col="neuron_type", value_col="value", round=1)
categorical_metrics = ["is_burst"]
df_res_categorical = (
    spiketrain_vol
    .query(f"metric in @categorical_metrics and value in (0, 1)")
    .groupby("metric")
    .apply(prop_calculator)
)

# display results
display(df_res_numeric)
display(df_res_categorical)

,F,ff - sir,ff - sr,sir - sr
metric,,,,
cv_isi,F(2)=3.8 (p=0.0*),0.2; 0.4 | T=-2.7 (p=0.0*),0.2; 0.4 | T=-2.4 (p=0.0*),0.4; 0.4 | T=0.3 (p=0.9)
cv_isi_burst,F(2)=4.1 (p=0.0*),0.2; 0.4 | T=-2.8 (p=0.0*),0.2; 0.4 | T=-2.3 (p=0.1),0.4; 0.4 | T=0.8 (p=0.7)
mean_firing_rate,F(2)=4.4 (p=0.0*),0.5; 0.8 | T=-2.9 (p=0.0*),0.5; 0.6 | T=-1.6 (p=0.2),0.8; 0.6 | T=1.7 (p=0.2)


,anova,ff - sir
metric,,
is_burst,Chi2(1)=0.2 (p=0.70),28.6%; 66.7% | Chi(1.0)=0.2 (p=0.7)
